# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [2]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [4]:
tracks = pd.read_parquet("./tracks.parquet")

In [5]:
catalog_names = pd.read_parquet("./catalog_names.parquet")

In [6]:
interactions = pd.read_parquet("./interactions.parquet")

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [7]:
tracks.info()
catalog_names.info()
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   int16         
 3   started_at  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), i

Пример данных

In [8]:
print(tracks[["track_id", "albums", "artists", "genres"]].sample(5, random_state=52).sort_values("track_id").set_index("track_id").to_string())
print(catalog_names[["id", "type", "name"]].sample(5, random_state=52).sort_values("id").set_index("id").to_string())
print(interactions[["user_id", "track_id", "track_seq", "started_at"]].sample(5, random_state=52).sort_values("user_id").set_index(["user_id", "track_id"]).to_string())

                          albums           artists        genres
track_id                                                        
88957     [8201, 88361, 6045574]            [3543]  [11, 44, 75]
46257838               [6233977]         [4383623]       [3, 75]
48263278      [6575614, 7058305]  [678505, 611169]          [68]
61200248               [9527234]          [915312]          [11]
97110155              [19968185]         [4172355]     [14, 102]
           type                      name
id                                       
87080     album                  El Cielo
4989459   album             Edm Club Hits
9088932   track  Let´s Make A Night Of It
37540174  track                Чертополох
69405644  track             A Lucid Dream
                  track_seq started_at
user_id track_id                      
46179   29544272         37 2022-11-23
70117   28213677       1055 2022-09-18
72819   85090862        270 2022-08-08
471152  55682            18 2022-09-10
688081  220497   

Смотрим на поля

In [9]:
print(tracks.sample(3).T)
print(catalog_names.sample(3).T)
print(interactions.sample(3).T)

             148623                              305635  \
track_id    3651054                            19782342   
albums     [407240]                           [2228702]   
artists    [442249]  [4510193, 743359, 590442, 3187610]   
genres    [68, 332]                             [3, 75]   

                            255917  
track_id                  15853252  
albums          [1728221, 1728222]  
artists   [95093, 1578515, 652473]  
genres                        [25]  
            655730  906904           156618
id        21019285  823778          2506340
type         album   track            album
name  Money So Big  Rialto  Hold In Silence
                            23                   592                  179
user_id                   95438               287565                59837
track_id               58381474             64261473             31170137
track_seq                    24                  593                  180
started_at  2022-11-08 00:00:00  2022-11-12 00:

Кол-во уникальных значений

In [10]:
print("Треки - ", tracks["track_id"].nunique())
print("Названия - ", catalog_names["id"].nunique())
print("Взаимодействия - ", interactions["user_id"].nunique())

Треки -  1000000
Названия -  1776697
Взаимодействия -  1373221


In [38]:
# Вычищаем треки с неизвесными исполнителями, альбомами, жанрами
# print(tracks.query("albums IN (1728221, 1728222)"))
unknown_ids = tracks[(tracks.artists.str.len() == 0) | (tracks.albums.str.len() == 0) | (tracks.genres.str.len() == 0)]["track_id"].to_list()
tracks = tracks.query("track_id != @unknown_ids")

print(tracks["track_id"].nunique())

980977


# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

Присутствовали данные без исполнителей или альбомов или жанров
Такие данные нам не подойдут для обучения, они вычищены из датасета

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

Наиболее популярные треки

Наиболее популярные жанры

Треки, которые никто не прослушал

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.